In [1]:
import re
import numpy as np
import pandas as pd
import pickle
import joblib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

# 사용 모델
from sklearn.ensemble import RandomForestClassifier
import lib.EncoderModel as mm

# 함수들 불러오기
import lib.functions as f

# 평가에 이용되는 함수 정의

In [2]:
# 1차 filter용 데이터셋 정의 -> (조문 + 조문제목) [embedding vector화 + 배치화]
class CustomDataset1(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['조문'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text_data = text1 + " // " + text2
        label = torch.tensor(self.data.iloc[idx]['사무판단'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [3]:
# 2차 filter용 데이터셋 정의 -> (법령명 + 조문제목 + 수행주체) [embedding vector화 + 배치화]
class CustomDataset2(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['법령명'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text3 = str(self.data.iloc[idx]['수행주체(문자열)'])
        text_data = text1 + " // " + text2 + " // " + text3
        label = torch.tensor(self.data.iloc[idx]['사무유형(대분류)'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [4]:
# 각 배치의 텍스트 길이를 맞추기
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # 패딩 적용
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': torch.stack(labels)}


In [5]:
# 평가 결과 확인
def cal_result(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    accuracy = accuracy_score(y, preds)
    precision = precision_score(y, preds)
    recall = recall_score(y, preds)
    f1 = f1_score(y, preds)
    
    # 결과 출력
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [6]:
def cal_result2(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    print('혼동행렬')
    print(conf_matrix)
    accuracy = accuracy_score(y, preds)
    print(f'정확도: {accuracy}')
    class_report = classification_report(y, preds)
    print('Classification Report:')
    print(class_report)
    
    return conf_matrix

# 1. 데이터 입력

In [7]:
test_df = pd.read_csv('data/FinalBaseTable.csv')
test_df.info()

/var/folders/zr/_f1rgf8n0w3541q9k8p3smhr0000gn/T/ipykernel_70046/1265917946.py:1: DtypeWarning: Columns (4,5,18,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('data/FinalBaseTable.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848072 entries, 0 to 848071
Data columns (total 26 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   소관부처명      848072 non-null  object
 1   법령명        848072 non-null  object
 2   법령구분       848072 non-null  int64 
 3   조번호        847493 non-null  object
 4   항번호        655168 non-null  object
 5   호번호        498265 non-null  object
 6   조문제목       848072 non-null  object
 7   조문         848072 non-null  object
 8   사무판단       848072 non-null  int64 
 9   사무판단근거     687920 non-null  object
 10  사무명        54588 non-null   object
 11  수행주체       54590 non-null   object
 12  사무유형       54546 non-null   object
 13  위임사무판단     848072 non-null  int64 
 14  위임근거규정     3749 non-null    object
 15  수임기관       2899 non-null    object
 16  특행기관       848072 non-null  int64 
 17  재위임사무판단    848072 non-null  int64 
 18  재위임근거규정    3 non-null       object
 19  재수임기관      2 non-null       object
 20  위탁사무

# 2. 데이터 전처리

In [8]:
#test_df = f.x_null_drop(test_df)
#test_df = f.x_wrong_drop(test_df)
test_df = f.no_work_check(test_df)
test_df = f.make_large_type(test_df)

In [9]:
# 결과 저장 데이터 프레임 생성
check_cols = ['법령명' ,'조문제목', '조문', '사무판단', '사무유형(대분류)']
sub_df = test_df.loc[:, check_cols].copy()
sub_df.shape

(848072, 5)

# 3. Rule-Base 적용

In [10]:
sub_df = f.rule_based(sub_df)  # rule_base 조문 확인

# rule_base로 걸러낸 조문들
rule_base_filter_idx = sub_df.loc[sub_df['rule_based']==0, :].index

# 4. 1차 사무판단
## encoder 모델

* encoder 구조 생성

In [11]:
# 모델 및 토크나이저 불러오기
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 기본 구조 설정
input_size = 512
hidden_size = 256
output_size = 2
num_encoder_layers = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = mm.MyModel(input_size, hidden_size, output_size, num_encoder_layers)
model.to(device)

MyModel(
  (encoder): MyEncoder(
    (layers): ModuleList(
      (0): MyEncoderLayer(
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (feedforward): Sequential(
          (0): Linear(in_features=512, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=512, bias=True)
        )
        (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (layer1): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer2): Linear(in_features=256, out_features=2, bias=True)
)

* 학습된 encoder 불러오기

In [12]:
model_path = 'pickles/encoder_model_f1_all.pth'  # Provide the correct path
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

* 모델 평가

In [13]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

test_dataset = CustomDataset1(sub_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [14]:
model.eval()

test_loss = 0
probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc=f'Validation'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = model(**inputs).float()
        test_loss += torch.nn.functional.cross_entropy(outputs, labels.long())
        
        # 소프트맥스를 적용하여 확률값 얻기
        probs_batch = torch.nn.functional.softmax(outputs, dim=1)
        probs.append(probs_batch.cpu().numpy())

# 손실 계산
avg_test_loss = test_loss / len(test_loader)

# 확률값을 하나의 배열로 통합
probs = np.concatenate(probs, axis=0)

print(avg_test_loss)

Validation: 100%|████████████████████████| 53005/53005 [04:32<00:00, 194.75it/s]


tensor(0.3663)


In [15]:
# 결과 확률 저장
prob_1 = probs[:, 1]
prob_1 = [round(prob, 5) for prob in prob_1]

In [16]:
sub_df.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based
0,개인정보 보호법,0,제1장 총칙,0,0,0
1,개인정보 보호법,목적,제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자...,0,0,0
2,개인정보 보호법,정의,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014.3.2...,0,0,0
3,개인정보 보호법,정의,"1. ""개인정보""란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당...",0,0,0
4,개인정보 보호법,정의,"1의2. ""가명처리""란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의...",0,0,0


## 1차 사무판단 진행

In [17]:
threshold = 0.251    # test 결과로 확인한 threshold

sub_df['사무판단 예측'] = 0
sub_df['사무판단 예측확률'] = prob_1

# 1차 사무판단
sub_df.loc[sub_df['사무판단 예측확률'] >= threshold, '사무판단 예측'] = 1

sub_df.loc[rule_base_filter_idx, '사무판단 예측'] = 0   # rule-base 결과 적용

In [18]:
#sub_df.loc[(sub_df['rule_based']==0) & (sub_df['사무판단 예측']==1), :]   # rule-base로 11425 추가 걸러냄

In [19]:
# 1차 필터링 결과
cal_result(sub_df['사무판단 예측'], sub_df['사무판단'])

Accuracy: 0.6776323236706318
Precision: 0.16125686866341615
Recall: 0.9549737835954973
F1-Score: 0.2759216147426577
Confusion Matrix:
[[522591 270935]
 [  2456  52090]]


## 1차 filtering 결과

In [20]:
# 1차 filter 결과
first_filtered_df = sub_df.loc[sub_df['사무판단 예측']==1, :].copy()
first_filtered_df.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
13,개인정보 보호법,개인정보 보호 원칙,① 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 ...,0,0,1,1,0.75711
14,개인정보 보호법,개인정보 보호 원칙,② 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 적합하게 개인정보를 처리...,0,0,1,1,0.79265
15,개인정보 보호법,개인정보 보호 원칙,"③ 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전...",0,0,1,1,0.68588
16,개인정보 보호법,개인정보 보호 원칙,④ 개인정보처리자는 개인정보의 처리 방법 및 종류 등에 따라 정보주체의 권리가 침해...,0,0,1,1,0.61160
17,개인정보 보호법,개인정보 보호 원칙,⑤ 개인정보처리자는 개인정보 처리방침 등 개인정보의 처리에 관한 사항을 공개하여야 ...,0,0,1,1,0.71247


# 4. 2차 사무판단 + 사무유형 판단
* 사무로 판단되었을때 실행

## Encoder Model

In [21]:
input_size = 128
hidden_size = 64
output_size = 4

total_encoder = mm.MyModel(input_size, hidden_size, output_size)

In [22]:
model_path2 = 'pickles/encoder_model_f2_all.pth'  # Provide the correct path
total_encoder.load_state_dict(torch.load(model_path2, map_location=device))

<All keys matched successfully>

* 수행주체 추출

In [23]:
# 수행주체 종류 불러오기
with open('pickles/subject_list.pkl', 'rb') as file:
    subject_list = pickle.load(file)

In [24]:
# 수행주체(문자열, 리스트 추가)
first_filtered_df = f.new_subject_make(first_filtered_df, subject_list, tqdm)

Processing rows: 100%|████████████████| 323025/323025 [00:50<00:00, 6360.19it/s]


In [25]:
first_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323025 entries, 13 to 848069
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   법령명         323025 non-null  object 
 1   조문제목        323025 non-null  object 
 2   조문          323025 non-null  object 
 3   사무판단        323025 non-null  int64  
 4   사무유형(대분류)   323025 non-null  int64  
 5   rule_based  323025 non-null  int64  
 6   사무판단 예측     323025 non-null  int64  
 7   사무판단 예측확률   323025 non-null  float32
 8   수행주체(리스트)   323025 non-null  object 
 9   수행주체(문자열)   323025 non-null  object 
dtypes: float32(1), int64(4), object(5)
memory usage: 33.9+ MB


In [26]:
first_filtered_df.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체(리스트),수행주체(문자열)
13,개인정보 보호법,개인정보 보호 원칙,① 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 ...,0,0,1,1,0.75711,[],
14,개인정보 보호법,개인정보 보호 원칙,② 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 적합하게 개인정보를 처리...,0,0,1,1,0.79265,[],
15,개인정보 보호법,개인정보 보호 원칙,"③ 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전...",0,0,1,1,0.68588,[],
16,개인정보 보호법,개인정보 보호 원칙,④ 개인정보처리자는 개인정보의 처리 방법 및 종류 등에 따라 정보주체의 권리가 침해...,0,0,1,1,0.61160,[관리],관리
17,개인정보 보호법,개인정보 보호 원칙,⑤ 개인정보처리자는 개인정보 처리방침 등 개인정보의 처리에 관한 사항을 공개하여야 ...,0,0,1,1,0.71247,[],


In [27]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

total_dataset = CustomDataset2(first_filtered_df, tokenizer)
total_loader = DataLoader(total_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [28]:
# 2차 인코더 가중치 학습
class_weights = [0.2969,  2.2733,  9.0880, 12.1487]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [29]:
total_encoder.eval()
val_loss = 0.0
y_true = []  # 실제 레이블을 저장할 리스트
y_pred = []  # 모델의 예측 결과를 저장할 리스트
y_probs = [] # 사무유형별 확률 저장할 리스트
with torch.no_grad():
    for batch in tqdm(total_loader, desc=f'Test'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = total_encoder(**inputs).float()
        
        probs = torch.nn.functional.softmax(outputs, dim=1)
        y_probs.extend(probs.cpu().numpy())
        _, predictions = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        val_loss += torch.nn.functional.cross_entropy(outputs, labels, weight=class_weights)

avg_val_loss = val_loss / len(total_loader)
print(f'Test Loss: {avg_val_loss}')

Test: 100%|██████████████████████████████| 20190/20190 [01:09<00:00, 289.14it/s]

Test Loss: 1.202365756034851


In [30]:
probs = pd.DataFrame(y_probs)
probs = probs.set_index(first_filtered_df.index)

# 인코더 결과 저장
encoder_result_df = f.make_result_df(pd, y_true, y_pred, probs[0], probs[1], probs[2], probs[3])

In [31]:
encoder_result_df

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률
13,0,0,0.48308,0.24405,0.10464,0.16823
14,0,0,0.48308,0.24405,0.10464,0.16823
15,0,0,0.48308,0.24405,0.10464,0.16823
16,0,0,0.47572,0.24771,0.10569,0.17089
17,0,0,0.48308,0.24405,0.10464,0.16823
...,...,...,...,...,...,...
848059,0,1,0.21159,0.40811,0.17917,0.20113
848060,0,1,0.21159,0.40811,0.17917,0.20113
848063,0,1,0.21159,0.40811,0.17917,0.20113
848066,0,1,0.28751,0.33557,0.24997,0.12695


In [32]:
# 결과 확인
cal_result2(encoder_result_df['사무유형(대분류) 예측'], encoder_result_df['사무유형(대분류)'])

혼동행렬
[[183486  63170  20707   3572]
 [ 19352  13009   3563    644]
 [  3803   3139   1834    193]
 [  2787   2475   1065    226]]
정확도: 0.6146737868586023
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.68      0.76    270935
           1       0.16      0.36      0.22     36568
           2       0.07      0.20      0.10      8969
           3       0.05      0.03      0.04      6553

    accuracy                           0.61    323025
   macro avg       0.29      0.32      0.28    323025
weighted avg       0.76      0.61      0.67    323025



array([[183486,  63170,  20707,   3572],
       [ 19352,  13009,   3563,    644],
       [  3803,   3139,   1834,    193],
       [  2787,   2475,   1065,    226]])

## Random Forest

In [33]:
rf_2_filtered_df = first_filtered_df.copy()

In [34]:
# RF 모델 불러오기
with open('pickles/rf_model_f2_all.pkl', 'rb') as file:
    model_2_rf = pickle.load(file)

In [35]:
rf_2_filtered_df.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체(리스트),수행주체(문자열)
13,개인정보 보호법,개인정보 보호 원칙,① 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 ...,0,0,1,1,0.75711,[],
14,개인정보 보호법,개인정보 보호 원칙,② 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 적합하게 개인정보를 처리...,0,0,1,1,0.79265,[],
15,개인정보 보호법,개인정보 보호 원칙,"③ 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전...",0,0,1,1,0.68588,[],
16,개인정보 보호법,개인정보 보호 원칙,④ 개인정보처리자는 개인정보의 처리 방법 및 종류 등에 따라 정보주체의 권리가 침해...,0,0,1,1,0.61160,[관리],관리
17,개인정보 보호법,개인정보 보호 원칙,⑤ 개인정보처리자는 개인정보 처리방침 등 개인정보의 처리에 관한 사항을 공개하여야 ...,0,0,1,1,0.71247,[],


In [36]:
# 법령명 통합
rf_2_filtered_df['법령명'] = rf_2_filtered_df['법령명'].apply(lambda x: re.sub(r'\s?시행령|\s?시행규칙', '', x))

In [37]:
# 새로운 컬럼을 이용하기 위한 수행주체 count dictionary
with open('pickles/subject_dic_f2_all.pkl', 'rb') as file:
    subject_dic = pickle.load(file)

In [38]:
# 수행주체 기준 새로운 열 생성
rf_2_filtered_df = f.score_subject(rf_2_filtered_df,subject_dic,tqdm)

Processing rows: 100%|████████████████| 323025/323025 [00:53<00:00, 6032.55it/s]


In [39]:
select_column = ['법령명', 'score_subject_nan', 'score_subject_n', 'score_subject_r', 'score_subject_p']
rf_2_filtered_label = rf_2_filtered_df['사무유형(대분류)']
rf_2_filtered_input = rf_2_filtered_df[select_column]

In [40]:
# '법령명' label encoding 진행
le = LabelEncoder()
rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform

/var/folders/zr/_f1rgf8n0w3541q9k8p3smhr0000gn/T/ipykernel_70046/253750670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform


In [41]:
# rf 결과 추출
y_pred_rf = model_2_rf.predict(rf_2_filtered_input)
y_prob_rf = model_2_rf.predict_proba(rf_2_filtered_input)

In [42]:
# rf 결과 데이터프레임 생성
rf_probs = pd.DataFrame(y_prob_rf)
rf_probs = rf_probs.set_index(first_filtered_df.index)

rf_result_df = f.make_result_df(pd, rf_2_filtered_label, y_pred_rf, rf_probs[0], rf_probs[1], rf_probs[2], rf_probs[3])

In [43]:
rf_result_df

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률
13,0,0,1.00000,0.00000,0.00000,0.0
14,0,0,1.00000,0.00000,0.00000,0.0
15,0,0,1.00000,0.00000,0.00000,0.0
16,0,0,1.00000,0.00000,0.00000,0.0
17,0,0,1.00000,0.00000,0.00000,0.0
...,...,...,...,...,...,...
848059,0,0,0.87963,0.00000,0.12037,0.0
848060,0,0,0.87963,0.00000,0.12037,0.0
848063,0,0,0.87963,0.00000,0.12037,0.0
848066,0,0,0.90212,0.09788,0.00000,0.0


In [44]:
# rf 결과 확인
cal_result2(rf_result_df['사무유형(대분류) 예측'], rf_result_df['사무유형(대분류)'])

혼동행렬
[[244390  19777   3820   2948]
 [ 20947  15178    132    311]
 [  6036    118   2728     87]
 [  3704    190     80   2579]]
정확도: 0.819982973454067
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90    270935
           1       0.43      0.42      0.42     36568
           2       0.40      0.30      0.35      8969
           3       0.44      0.39      0.41      6553

    accuracy                           0.82    323025
   macro avg       0.54      0.50      0.52    323025
weighted avg       0.81      0.82      0.82    323025



array([[244390,  19777,   3820,   2948],
       [ 20947,  15178,    132,    311],
       [  6036,    118,   2728,     87],
       [  3704,    190,     80,   2579]])

## Ensemble

In [45]:
pred = [0]*len(y_prob_rf)
e_prob0 = (rf_result_df['비사무 확률'] + encoder_result_df['비사무 확률'])/2
e_prob1 = (rf_result_df['국가 확률'] + encoder_result_df['국가 확률'])/2
e_prob2 = (rf_result_df['지방 확률'] + encoder_result_df['지방 확률'])/2
e_prob3 = (rf_result_df['공동 확률'] + encoder_result_df['공동 확률'])/2

ensemble_result_df_11 = f.make_result_df(pd, rf_2_filtered_label, pred, e_prob0, e_prob1, e_prob2, e_prob3)
ensemble_result_df_11['수행주체(리스트)'] = rf_2_filtered_df['수행주체(리스트)']

In [46]:
# 앙상블 사무유형 판단 저장
ensemble_result_df_11 = f.make_ensemble_pred(ensemble_result_df_11)
ensemble_result_df_11['사무유형(대분류) 예측'] = ensemble_result_df_11['사무유형(대분류) 예측'].astype(int)
ensemble_result_df_11['rule_based'] = first_filtered_df['rule_based']

In [47]:
# 앙살블 결과 확인
cal_result2(ensemble_result_df_11['사무유형(대분류) 예측'], ensemble_result_df_11['사무유형(대분류)'])

혼동행렬
[[247496  17780   3260   2399]
 [ 21934  14265    100    269]
 [  6234    123   2538     74]
 [  3868    201     85   2399]]
정확도: 0.825626499496943
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90    270935
           1       0.44      0.39      0.41     36568
           2       0.42      0.28      0.34      8969
           3       0.47      0.37      0.41      6553

    accuracy                           0.83    323025
   macro avg       0.55      0.49      0.52    323025
weighted avg       0.81      0.83      0.82    323025



array([[247496,  17780,   3260,   2399],
       [ 21934,  14265,    100,    269],
       [  6234,    123,   2538,     74],
       [  3868,    201,     85,   2399]])

In [48]:
# 확실/주의 나누기 [확실: need_to_check = 0 / 주의: need_to_check = 1]
ensemble_result_df_11, good_df, bad_df = f.make_need_to_check(ensemble_result_df_11,
                                                           rf_result_df, encoder_result_df)

In [49]:
ensemble_result_df_11.head()

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
13,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
14,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
15,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
16,0,0,0.73786,0.12386,0.05285,0.08545,[관리],1,0
17,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0


In [50]:
# 확실 결과
cal_result2(good_df['사무유형(대분류) 예측'].tolist(), good_df['사무유형(대분류)'].tolist())

혼동행렬
[[168564   6528    643     88]
 [ 10940   5485     14      7]
 [  2619     34    624      1]
 [  1529     78     11     86]]
정확도: 0.8859726946884934
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94    175823
           1       0.45      0.33      0.38     16446
           2       0.48      0.19      0.27      3278
           3       0.47      0.05      0.09      1704

    accuracy                           0.89    197251
   macro avg       0.58      0.38      0.42    197251
weighted avg       0.87      0.89      0.87    197251



array([[168564,   6528,    643,     88],
       [ 10940,   5485,     14,      7],
       [  2619,     34,    624,      1],
       [  1529,     78,     11,     86]])

In [51]:
# 애매 결과
cal_result2(bad_df['사무유형(대분류) 예측'].tolist(), bad_df['사무유형(대분류)'].tolist())

혼동행렬
[[78932 11252  2617  2311]
 [10994  8780    86   262]
 [ 3615    89  1914    73]
 [ 2339   123    74  2313]]
정확도: 0.7309857363207022
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83     95112
           1       0.43      0.44      0.44     20122
           2       0.41      0.34      0.37      5691
           3       0.47      0.48      0.47      4849

    accuracy                           0.73    125774
   macro avg       0.53      0.52      0.53    125774
weighted avg       0.73      0.73      0.73    125774



array([[78932, 11252,  2617,  2311],
       [10994,  8780,    86,   262],
       [ 3615,    89,  1914,    73],
       [ 2339,   123,    74,  2313]])

## 2차 filter 결과 저장

In [52]:
sencond_filtered_df = ensemble_result_df_11.copy()   # 2차 필터 결과 저장

In [53]:
# 2차 filter 결과 확인
cal_result2(sencond_filtered_df['사무유형(대분류) 예측'], sencond_filtered_df['사무유형(대분류)'])

혼동행렬
[[247496  17780   3260   2399]
 [ 21934  14265    100    269]
 [  6234    123   2538     74]
 [  3868    201     85   2399]]
정확도: 0.825626499496943
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90    270935
           1       0.44      0.39      0.41     36568
           2       0.42      0.28      0.34      8969
           3       0.47      0.37      0.41      6553

    accuracy                           0.83    323025
   macro avg       0.55      0.49      0.52    323025
weighted avg       0.81      0.83      0.82    323025



array([[247496,  17780,   3260,   2399],
       [ 21934,  14265,    100,    269],
       [  6234,    123,   2538,     74],
       [  3868,    201,     85,   2399]])

In [54]:
# 2차 filter (확실/주의)결과 확인
h_ratio, j_ratio = f.check_result(sencond_filtered_df)

주의 사무 개수: 125774
주의에서 틀린 개수: 33835
주의에서 맞은 개수: 91939
------------------
확실 사무 개수: 197251
확실에서 틀린 개수: 22492
확실에서 맞은 개수: 174759
확실 정확도: 0.8859726946884934
애매 정확도: 0.7309857363207022


In [55]:
sencond_filtered_df

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
13,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
14,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
15,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
16,0,0,0.73786,0.12386,0.05285,0.08545,[관리],1,0
17,0,0,0.74154,0.12202,0.05232,0.08411,[],1,0
...,...,...,...,...,...,...,...,...,...
848059,0,0,0.54561,0.20405,0.14977,0.10057,"[사업자, 수도사업자, 일반수도사업자, 지방환경관서]",1,1
848060,0,0,0.54561,0.20405,0.14977,0.10057,"[사업자, 수도사업자, 일반수도사업자, 지방환경관서]",1,1
848063,0,0,0.54561,0.20405,0.14977,0.10057,"[사업자, 수도사업자, 일반수도사업자, 지방환경관서]",1,1
848066,0,0,0.59482,0.21673,0.12499,0.06347,"[기관, 장관, 전문기관, 지방환경관서, 환경부]",1,1


# 최종 결과 확인

In [56]:
sub_df.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
0,개인정보 보호법,0,제1장 총칙,0,0,0,0,0.00439
1,개인정보 보호법,목적,제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자...,0,0,0,0,0.59747
2,개인정보 보호법,정의,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014.3.2...,0,0,0,0,0.07633
3,개인정보 보호법,정의,"1. ""개인정보""란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당...",0,0,0,0,0.02239
4,개인정보 보호법,정의,"1의2. ""가명처리""란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의...",0,0,0,0,0.26156


In [57]:
# 최종 결과 저장
sub_df = f.make_final_df(sub_df, sencond_filtered_df)
sub_df

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체,사무유형(대분류) 결과,비사무 확률,국가 확률,지방 확률,공동 확률,need_to_check
0,개인정보 보호법,0,제1장 총칙,0,0,0.0,0,0.00439,,0,0.00000,0.0000,0.00000,0.00000,0
1,개인정보 보호법,목적,제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자...,0,0,0.0,0,0.59747,,0,0.00000,0.0000,0.00000,0.00000,0
2,개인정보 보호법,정의,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014.3.2...,0,0,0.0,0,0.07633,,0,0.00000,0.0000,0.00000,0.00000,0
3,개인정보 보호법,정의,"1. ""개인정보""란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당...",0,0,0.0,0,0.02239,,0,0.00000,0.0000,0.00000,0.00000,0
4,개인정보 보호법,정의,"1의2. ""가명처리""란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의...",0,0,0.0,0,0.26156,,0,0.00000,0.0000,0.00000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848067,수도법 시행규칙,기술진단 결과의 평가,1. 한국수자원공사,0,0,0.0,0,0.00781,,0,0.00000,0.0000,0.00000,0.00000,0
848068,수도법 시행규칙,기술진단 결과의 평가,2. 법 제56조에 따른 한국상하수도협회,0,0,0.0,0,0.00247,,0,0.00000,0.0000,0.00000,0.00000,0
848069,수도법 시행규칙,규제의 재검토,제32조(규제의 재검토) 환경부장관은 다음 각 호의 사항에 대하여 다음 각 호의 기...,0,0,1.0,1,0.61423,"[장관, 환경부]",0,0.79397,0.0933,0.05175,0.06098,0
848070,수도법 시행규칙,규제의 재검토,1. 제23조의2제1항ㆍ제4항 및 별표 7의2에 따른 저수조청소업의 인력ㆍ시설 및 ...,0,0,0.0,0,0.02749,,0,0.00000,0.0000,0.00000,0.00000,0


## 사무판단 정오 추가

In [58]:
ex = sub_df.copy()
ex.loc[ex['사무유형(대분류) 결과']!=0, '사무유형(대분류) 결과'] = 1

In [59]:
# 정오표 생성
sub_df['사무판단_정오'] = 0
sub_df.loc[(sub_df['사무판단']==0)&(sub_df['사무판단']==ex['사무유형(대분류) 결과']), '사무판단_정오'] = 0
sub_df.loc[(sub_df['사무판단']==0)&(sub_df['사무판단']!=ex['사무유형(대분류) 결과']), '사무판단_정오'] = 1
sub_df.loc[(sub_df['사무판단']==1)&(sub_df['사무판단']!=ex['사무유형(대분류) 결과']), '사무판단_정오'] = 2
sub_df.loc[(sub_df['사무판단']==1)&(sub_df['사무판단']==ex['사무유형(대분류) 결과']), '사무판단_정오'] = 3

In [60]:
sub_df['사무판단_정오'].value_counts()

0    770087
2     34492
1     23439
3     20054
Name: 사무판단_정오, dtype: int64

In [61]:
sub_df.to_csv('data/final_df(total1).csv', index=False)

## 사무유형 예측 top 2 결과 확인

In [62]:
ex = sub_df.copy()
ex['top2 결과'] = 0
for i in range(len(ex)):
    l = list(ex.loc[i, '비사무 확률':'공동 확률'])
    s = set(l)
    if len(s) == 1:
        continue
    sorted_lst = sorted(l)
    second_largest = sorted_lst[-2]
    second_largest_index = l.index(second_largest)
    ex.loc[i, 'top2 결과'] = second_largest_index

In [63]:
ex['top2 정답'] = 0
idxs = (ex['사무유형(대분류)']==ex['사무유형(대분류) 결과'])|(ex['사무유형(대분류)']==ex['top2 결과'])
ex.loc[idxs ,'top2 정답'] = ex.loc[idxs ,'사무유형(대분류)']

cal_result2(ex['top2 정답'], ex['사무유형(대분류)'])

혼동행렬
[[793526      0      0      0]
 [  4074  34413      0      0]
 [  3135      0   6250      0]
 [  2465      0      0   4209]]
정확도: 0.9885929496552179
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    793526
           1       1.00      0.89      0.94     38487
           2       1.00      0.67      0.80      9385
           3       1.00      0.63      0.77      6674

    accuracy                           0.99    848072
   macro avg       1.00      0.80      0.88    848072
weighted avg       0.99      0.99      0.99    848072



array([[793526,      0,      0,      0],
       [  4074,  34413,      0,      0],
       [  3135,      0,   6250,      0],
       [  2465,      0,      0,   4209]])

In [64]:
concatenated_df = pd.concat([test_df, sub_df.iloc[:, 5:]], axis=1)
concatenated_df.head(1)

,소관부처명,법령명,법령구분,조번호,항번호,호번호,조문제목,조문,사무판단,사무판단근거,...,사무판단 예측,사무판단 예측확률,수행주체,사무유형(대분류) 결과,비사무 확률,국가 확률,지방 확률,공동 확률,need_to_check,사무판단_정오
0,개인정보보호위원회,개인정보 보호법,1,1,0,0,0,제1장 총칙,0,NaN,...,0,0.00439,,0,0.0,0.0,0.0,0.0,0,0


In [66]:
ex = sub_df.copy()
ex.loc[ex['사무유형(대분류) 결과']!=0, '사무유형(대분류) 결과'] = 1

cal_result(ex['사무유형(대분류) 결과'], ex['사무판단'])

Accuracy: 0.9316909413351696
Precision: 0.4610856919504288
Recall: 0.3676529901367653
F1-Score: 0.4091025000255001
Confusion Matrix:
[[770087  23439]
 [ 34492  20054]]


In [67]:
cal_result2(sub_df['사무유형(대분류) 결과'], sub_df['사무유형(대분류)'])

혼동행렬
[[770087  17780   3260   2399]
 [ 23853  14265    100    269]
 [  6650    123   2538     74]
 [  3989    201     85   2399]]
정확도: 0.930686309652954
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    793526
           1       0.44      0.37      0.40     38487
           2       0.42      0.27      0.33      9385
           3       0.47      0.36      0.41      6674

    accuracy                           0.93    848072
   macro avg       0.57      0.49      0.53    848072
weighted avg       0.92      0.93      0.93    848072



array([[770087,  17780,   3260,   2399],
       [ 23853,  14265,    100,    269],
       [  6650,    123,   2538,     74],
       [  3989,    201,     85,   2399]])

In [65]:
concatenated_df.to_csv('data/final_df(total2).csv', index=False)